In [1]:
import torch as t
import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint
from tqdm import tqdm, trange
import einops
from dataclasses import dataclass
from typing import List
from functools import partial
import plotly.express as px

if t.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if t.cuda.is_available() else "cpu"

# Clamping SAE features in one layer and measuring effects on the subsequent layer
## Plan described [here](https://spar2024.slack.com/archives/C0794GNT8KS/p1719950740186749?thread_ts=1719934219.491869&cid=C0794GNT8KS)

In [2]:
# loading a small set of correlations to play around with
pearson_0_1_small: 'f' = t.load('../../data/res_jb_sae_feature_correlation_pearson_0_1.pt')

In [3]:
# find the highest correlations
def create_value_tensor(matrix: 'f,f') -> 'f*f,3':
    m, _ = matrix.shape
    
    # Step 1: Flatten the matrix (shape: [m*m])
    flattened_matrix = matrix.flatten()
    
    # Step 2: Create row and column indices
    row_indices = t.arange(m).repeat_interleave(m)
    col_indices = t.arange(m).repeat(m)
    
    # Step 3: Create the final tensor with indices and values
    values = flattened_matrix
    result = t.stack((row_indices, col_indices, values), dim=1)
    
    # Step 4: Sort the result tensor by values
    sorted_result = result[t.argsort(result[:, 2], descending=True)]
    
    return sorted_result

In [10]:
pearson_0_1_small

tensor([[-2.9526e-05, -4.4647e-05, -9.5376e-05,  ..., -2.0311e-05,
         -1.4521e-05, -5.3107e-05],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [-1.6616e-04, -2.5125e-04, -5.3673e-04,  ..., -1.1430e-04,
         -8.1718e-05,  3.1849e-04],
        ...,
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [-1.5595e-04, -2.3582e-04, -5.0376e-04,  ..., -1.0728e-04,
         -7.6699e-05, -2.8050e-04]])

In [4]:
ranked_features = create_value_tensor(pearson_0_1_small)

In [38]:
# Create a mask to identify rows where the last column is not NaN
not_nan_mask = ~torch.isnan(ranked_features[:, 2])

# Create a mask to identify rows where the last column is less than 1
less_than_one_mask = ranked_features[:, 2] < 0.99

# Combine masks using logical AND
combined_mask = not_nan_mask & less_than_one_mask

# Use the combined mask to filter the rows
filtered_tensor = ranked_features[combined_mask]

In [39]:
filtered_tensor

tensor([[ 1.0715e+04,  2.0175e+04,  9.8999e-01],
        [ 1.8389e+04,  5.4670e+03,  9.8998e-01],
        [ 6.2600e+03,  1.3641e+04,  9.8998e-01],
        ...,
        [ 1.3725e+04,  9.4870e+03, -1.4345e-01],
        [ 1.3725e+04,  1.5370e+04, -1.4875e-01],
        [ 1.3725e+04,  2.2119e+04, -1.5495e-01]])

In [41]:
pearson_0_1_small[10715, 20175]

tensor(0.9900)

## Measure the correlation when we pass the residual stream that's reconstructed from the SAE features - NO clamping
As mentioned at the end of June, normally the SAE features values are read by us and discarded - they are not passed back into the model for inference.
However, if we are going to be clamping an SAE feature and seeing its impact downstream, then we need to first see what happens when we pass the SAE features downstream with no clamping - because the mere act of projecting a residual stream into an SAE space and then back into residual stream space is a lossy operation (even though the SAE is supposed to represent the residual stream)

In [42]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)
sae_id_to_sae = {}
for layer in tqdm(list(range(model.cfg.n_layers))):
    sae_id = f"blocks.{layer}.hook_resid_pre"
    sae, _, _ = SAE.from_pretrained(
        release="gpt2-small-res-jb",
        sae_id=sae_id,
        device=device
    )
    sae.eval()  # prevents error if we're expecting a dead neuron mask for who grads
    sae_id_to_sae[sae_id] = sae

/Users/benlerner/work/spar-2024/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


  0%|          | 0/12 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 12/12 [00:08<00:00,  1.34it/s]


In [43]:
from transformer_lens.utils import tokenize_and_concatenate
from datasets import load_dataset
from torch.utils.data import DataLoader
# These hyperparameters are used to pre-process the data
pre_0_sae_id = "blocks.0.hook_resid_pre"
pre_0_sae = sae_id_to_sae[pre_0_sae_id]
context_size = pre_0_sae.cfg.context_size
prepend_bos = pre_0_sae.cfg.prepend_bos
d_sae = pre_0_sae.cfg.d_sae
batch_size = 32

dataset = load_dataset(path="NeelNanda/pile-10k", split="train", streaming=False)
token_dataset = tokenize_and_concatenate(
    dataset=dataset,  # type: ignore
    tokenizer=model.tokenizer,  # type: ignore
    streaming=True,
    max_length=context_size,
    add_bos_token=prepend_bos,
)

tokens = token_dataset['tokens']

In [44]:
# OPTIONAL: Reduce dataset for faster experimentation
tokens = tokens[:1024]

In [45]:
data_loader = DataLoader(tokens, batch_size=batch_size, shuffle=False)

In [62]:
# looking at compute-pearson-0.py to figure out how hooks work


# okay so add_hook takes a function...and then...
# https://transformerlensorg.github.io/TransformerLens/generated/code/transformer_lens.hook_points.html#transformer_lens.hook_points.HookPoint.add_hook

# TODO: make a lambda function
# model.reset_hooks()

# TODO: find out where "pre" is defined
# michael: every time you do something with an activation. might not mean anything bc "residual stream" is not an action (unlike, say, adding the attention to the residual)
# https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/full-merm.svg
# model.add_hook("blocks.0.hook_resid_pre", replace_with_sae_output)

Okay...nice...looks like most of the output logits have changed now $$that this single layer is using SAE activations instead of the residual stream!

Next step: checking the correlation of 55, 4

how do I do that...
okay well how did we compute correlations initially?

First I need to collect the sae_activations but this time with the SAE being used to influence the second layer's activations

In [46]:
@dataclass
class LayerFeatures:
    layer_idx: int
    feature_idxes: List[int]

@dataclass
class AggregatorConfig:
    layer_1: LayerFeatures
    layer_2: LayerFeatures
    
class BatchedPearson:
    def __init__(self, agg_conf: AggregatorConfig):
        """Calculates the pair-wise Pearson correlation of two tensors that are provided batch-wise.
        """
        self.agg_conf = agg_conf
        shape = (len(agg_conf.layer_1.feature_idxes), len(agg_conf.layer_2.feature_idxes))
        self.count = 0

        self.sums_1 = torch.zeros(shape[0])
        self.sums_2 = torch.zeros(shape[1])

        self.sums_of_squares_1 = torch.zeros(shape[0])
        self.sums_of_squares_2 = torch.zeros(shape[1])

        self.sums_1_2 = torch.zeros(shape)

        self.nonzero_counts_1 = torch.zeros(shape[0])
        self.nonzero_counts_2 = torch.zeros(shape[1])

    def process(self, tensor_1, tensor_2):
        self.count += tensor_1.shape[-1]

        self.sums_1 += tensor_1.sum(dim=-1)
        self.sums_2 += tensor_2.sum(dim=-1)

        self.sums_of_squares_1 += (tensor_1 ** 2).sum(dim=-1)
        self.sums_of_squares_2 += (tensor_2 ** 2).sum(dim=-1)

        self.sums_1_2 += einops.einsum(tensor_1, tensor_2, 'f1 t, f2 t -> f1 f2')

        self.nonzero_counts_1 += tensor_1.count_nonzero(dim=-1)
        self.nonzero_counts_2 += tensor_2.count_nonzero(dim=-1)

    def finalize(self):
        means_1 = self.sums_1 / self.count
        means_2 = self.sums_2 / self.count

        # Compute the covariance and variances
        covariances = (self.sums_1_2 / self.count) - einops.einsum(means_1, means_2, 'f1, f2 -> f1 f2')

        variances_1 = (self.sums_of_squares_1 / self.count) - (means_1 ** 2)
        variances_2 = (self.sums_of_squares_2 / self.count) - (means_2 ** 2)

        stds_1 = torch.sqrt(variances_1).unsqueeze(1)
        stds_2 = torch.sqrt(variances_2).unsqueeze(0)

        # Compute the Pearson correlation coefficient
        correlations = covariances / stds_1 / stds_2

        return correlations

In [47]:
agg_confs = [
    AggregatorConfig(
        LayerFeatures(0, [10715]),
        LayerFeatures(1, [20175]),
    ),
]
aggs = [BatchedPearson(agg_conf) for agg_conf in agg_confs]
sae_activations = torch.empty(model.cfg.n_layers, d_sae, batch_size * context_size)

def replace_acts_with_sae(activations: t.Tensor, hook: HookPoint):
    # replaces the residual stream activations with SAE activations
    sae = sae_id_to_sae[hook.name]
    return sae(activations)

def emit_sae(activations: t.Tensor, hook: HookPoint):
    # emits what the SAE activations would be for these input activations
    sae: SAE = sae_id_to_sae[hook.name]
    sae_acts = sae.encode(activations)
    sae_activations[hook.layer()] = einops.rearrange(
        sae_acts,
        'batch seq features -> features (batch seq)'
    )
    return activations

with torch.no_grad():
    for batch_tokens in tqdm(data_loader):
        # okay so if we have a hook which is doing SAE embeddings and returning them then...we need to be...
        # i think it'll be easier if BOTH functions save SAEs, but only one ofthem returns the SAEs and other returns original acts
        model.reset_hooks()
        # TODO(optimization): collapse these into a single function which decides whether to 
        # emit the SAE activations or return the originals
        model.add_hook(
            lambda name: name.endswith('.hook_resid_pre'),
            emit_sae,
        )
        model.run_with_hooks(batch_tokens)
        for agg in aggs:
            agg.process(
                sae_activations[agg.agg_conf.layer_1.layer_idx, agg.agg_conf.layer_1.feature_idxes],
                sae_activations[agg.agg_conf.layer_2.layer_idx, agg.agg_conf.layer_2.feature_idxes]
            )
    pearson_correlations = [aggregator.finalize() for aggregator in aggs]

100%|██████████| 32/32 [01:25<00:00,  2.66s/it]


In [ ]:
pearson_correlations[]

In [82]:
ranked_feats = t.concat([
    create_value_tensor(pc)
    for pc in pearson_correlations
])

In [50]:
#
pearson_correlations[0]

tensor([[0.9900]])

# That's what I suspected
Now I'm going to recompute it but I'll see what happens when I feed the lossy first layer into the second layer

In [51]:
agg_confs = [
    AggregatorConfig(
        LayerFeatures(0, [10715]),
        LayerFeatures(1, [20175]),
    ),
]
aggs = [BatchedPearson(agg_conf) for agg_conf in agg_confs]
sae_activations = torch.empty(model.cfg.n_layers, d_sae, batch_size * context_size)

def replace_acts_with_lossy_sae(activations: t.Tensor, hook: HookPoint):
    # replaces the residual stream activations with SAE activations
    sae = sae_id_to_sae[hook.name]
    return sae(activations)

def emit_sae(activations: t.Tensor, hook: HookPoint):
    # emits what the SAE activations would be for these input activations
    sae: SAE = sae_id_to_sae[hook.name]
    sae_acts = sae.encode(activations)
    sae_activations[hook.layer()] = einops.rearrange(
        sae_acts,
        'batch seq features -> features (batch seq)'
    )
    return activations

with torch.no_grad():
    for batch_tokens in tqdm(data_loader):
        # okay so if we have a hook which is doing SAE embeddings and returning them then...we need to be...
        # i think it'll be easier if BOTH functions save SAEs, but only one ofthem returns the SAEs and other returns original acts
        model.reset_hooks()
        # TODO(optimization): collapse these into a single function which decides whether to 
        # emit the SAE activations or return the originals
        model.add_hook(
            lambda name: name.endswith('.hook_resid_pre'), # we emit the SAE activations no matter what
            emit_sae,
        )
        model.add_hook(
            "blocks.0.hook_resid_pre", # TODO: is this right?? is this just a token + positional embedding??
            replace_acts_with_lossy_sae,
        )
        model.run_with_hooks(batch_tokens)
        for agg in aggs:
            agg.process(
                sae_activations[agg.agg_conf.layer_1.layer_idx, agg.agg_conf.layer_1.feature_idxes],
                sae_activations[agg.agg_conf.layer_2.layer_idx, agg.agg_conf.layer_2.feature_idxes]
            )
    pearson_correlations = [aggregator.finalize() for aggregator in aggs]

100%|██████████| 32/32 [01:46<00:00,  3.33s/it]


In [52]:
pearson_correlations[0]

tensor([[0.9909]])

Hmmm...they're even *more* correlated.
Is that what I would expect? Maybe...if now, the layer1 feature value is lower on avg, so it would be more "in line" with layer 0?

## Extending this to work on multiple feature pairs within a single layer pair
TODO

## Extending this to work across multiple layer pairs
TODO